In [1]:
# Install necessary packages
!pip install llama-cpp-python

import os
from langchain.llms import LlamaCpp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=4552822 sha256=e7923fb84af71f6ee2540df1dbf4b8628d4025cb49e8d39e2db9b3bd92a00190
  Stored in directory: /root/.cache/pip/wheels/eb/82/79/ac77fcd49324b75ae6aa18e63a87cf9da4371a57e2cdc8dc03
Successfully built llama-cpp-python


ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
pip install -U langchain-openai

In [ ]:
!pip install mysql-connector-python sqlalchemy

In [ ]:
!pip install tavily-python


In [ ]:
!pip install langgraph

# **Add History to Prompt**
Process: Collect previous Q&A context and append it to the current user query to enhance the LLM’s understanding for follow-up questions.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import HumanMessage, AIMessage

# Initialize ConversationBufferWindowMemory with a window size (k) of 5 interactions
memory = ConversationBufferWindowMemory(memory_key="chat_history", k=5, return_messages=True)

def update_conversation_memory(memory, human_input: str, ai_response: str):
    """
    Updates the ConversationBufferWindowMemory with a new conversation turn.

    Args:
        memory: The ConversationBufferWindowMemory instance.
        human_input: The new user prompt.
        ai_response: The AI's response corresponding to the prompt.

    Returns:
        Updated memory containing the new turn.
    """
    # Add the new user message and AI response to the memory's chat_history.
    memory.chat_memory.add_message(HumanMessage(content=human_input))
    memory.chat_memory.add_message(AIMessage(content=ai_response))
    return memory

# Example usage:
# Simulate a conversation turn:
human_input = "Hello, how are you today?"
ai_response = "I'm doing well, thank you! How can I assist you?"
memory = update_conversation_memory(memory, human_input, ai_response)

# Retrieve the current conversation history as a string (or list of messages)
current_history = memory.load_memory_variables({})["chat_history"]
print("Current Conversation History:\n", current_history)


# **Router Decision Process**
Process: Analyze the user query to decide whether it should be processed via internal database querying or external web search (e.g., using keywords like “sales” or “disaster”).

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
openai_api_key = ""

# Initialize the LLM (using a deterministic setting)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0,openai_api_key=openai_api_key)

def decide_query_type(query: str) -> str:
    """
    Decides whether a query should be processed via an internal SQL query (use_db)
    or via external web search (use_web) based on the provided database schema.
    """
    # Define the schema context based on your table schematics:
    schema = """
    Database Schema:
    1. Users (Users): Id (PK), Name, Email (UK), PhoneNumber.
    2. NGOs (Ngos): ID (PK), Name, Code (UK).
    3. Campaigns (Campaigns): ID (PK), Title, NgoID (FK → Ngos(ID)), CampaignerId (FK → Users(Id)), Status.
    4. Categories (Category): Id (PK), Name, Priority.
    5. Campaign Categories (CampaignCategory): CampaignId (PK, FK → Campaigns(ID)), CategoryId (PK, FK → Category(Id)), Priority.
    6. Campaign Orders (CampaignOrders): Id (PK), CampaignId (FK → Campaigns(ID)), TransactionReference (FK → Transactions(Reference)), Amount.
    7. Transactions (Transactions): Reference (PK, UK), UserID (FK → Users(Id)), Amount, Status.
    """

    # Create a prompt template that instructs the LLM to decide the routing
    prompt_template = PromptTemplate(
        input_variables=["schema", "query"],
        template="""
You are an expert in database querying.
Given the following database schema:
{schema}

And the user query: "{query}"

Decide whether this query is intended for data retrieval from the database or if it is a general question better answered using external web search.
If the query refers to data that exists in the tables (for example, asking for totals, counts, or other metrics related to campaigns, orders, etc.), respond with "use_db". Otherwise, respond with "use_web".
        """
    )

    # Format the prompt with the schema and query
    prompt = prompt_template.format(schema=schema, query=query)

    # Get the LLM's decision
    decision = llm.predict(prompt)

    # Clean up the output (expecting a short response "use_db" or "use_web")
    decision = decision.strip().lower()
    if "use_db" in decision:
        return "use_db"
    else:
        return "use_web"

# Example usage:
queries = [
    "How is a llm model?",
    "What is the total revenue for campaign orders?",
    "Tell me about the latest global technology news.",
    "What are the total transactions for the last month?"
]

for q in queries:
    route = decide_query_type(q)
    print(f"Query: {q}\nRouted to: {route}\n")


# **Fetch Schema Process**
Process: Dynamically extract relevant schema metadata from your MySQL database (via Google Cloud RDS). This includes retrieving table names, columns, and foreign key relationships of all the schemas in a LLM friendly way.

In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy.sql import text  # Import text for executing raw SQL queries
import urllib.parse

# ✅ Encode password correctly for MySQL connection
user = "aadish"
password = urllib.parse.quote("Puiya@4369")  # Encodes special characters
host = "34.59.101.173"
port = "3306"
database = "test_schema"

# ✅ Create the database connection
db = SQLDatabase.from_uri(f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}")

def fetch_schema_metadata():
    """
    Dynamically fetches schema metadata including:
    - Table names
    - Column details
    - Foreign key relationships
    """
    # ✅ Retrieve table information
    table_info = db.get_table_info()

    # ✅ Query to get foreign key relationships
    foreign_key_query = text("""
        SELECT
            TABLE_NAME, COLUMN_NAME, CONSTRAINT_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME
        FROM
            INFORMATION_SCHEMA.KEY_COLUMN_USAGE
        WHERE
            TABLE_SCHEMA = :database AND REFERENCED_TABLE_NAME IS NOT NULL;
    """)

    # ✅ Execute query to get foreign key relationships
    with db._engine.connect() as connection:
        result = connection.execute(foreign_key_query, {"database": database})
        foreign_key_relations = result.fetchall()

    # ✅ Structure foreign key relationships
    foreign_keys = []
    for row in foreign_key_relations:
        foreign_keys.append({
            "table": row[0],
            "column": row[1],
            "constraint": row[2],
            "referenced_table": row[3],
            "referenced_column": row[4]
        })

    return {
        "tables": table_info,
        "foreign_keys": foreign_keys
    }

# ✅ Example Usage
schema_metadata = fetch_schema_metadata()
print(schema_metadata)

# **Generate SQL Context for efficient Query (RAG-based)**
Process: Using the retrieved schema  and the NLP query to return Effiient SQL context.

In [ ]:
from langchain_core.pydantic_v1 import BaseModel
from typing import List, Dict

class TableDetailsSchema(BaseModel):
    Tables: List[str]
    Attributes: Dict[str, List[str]]
    FK_Relationships: str


In [ ]:
import os
from typing import Dict, List, Optional, Set
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model

# Define the schema using Pydantic.
class TableDetailsSchema(BaseModel):
    """Schema for required tables, attributes, and relationships."""
    tables_required: List[str] = Field(..., description="List of required table names")
    attributes_required: Optional[Dict[str, Set[str]]] = Field(
        None,
        description=(
            "Dictionary mapping table names to required attributes. "
            "For example: {'users': {'id', 'name'}}"
        )
    )
    relationships: List[str] = Field(..., description="List of foreign key relationships")


llm = init_chat_model("gpt-4o", model_provider="openai", temperature=0, openai_api_key=openai_api_key)
structured_llm = llm.with_structured_output(TableDetailsSchema, method="function_calling")

def get_table_details(nl_query: str) -> Dict:
    """
    Uses the schema metadata and the natural language query to return a structured JSON output.

    Args:
        nl_query (str): The natural language query describing the data requirements.

    Returns:
        dict: A structured response listing required tables, attributes, and relationships.
    """

    schema_metadata = fetch_schema_metadata()

    input_data = (
        f"Database Schema:\n\nTables Definition:\n{schema_metadata['tables']}\n\n"
        f"Foreign Keys:\n{schema_metadata['foreign_keys']}\n\n"
        f"User Query:\n{nl_query}"
    )
    structured_output = structured_llm.invoke(input_data)

    result_dict = structured_output.model_dump()

    return result_dict


In [ ]:
nl_query = "which was the highest transaction users"
result = get_table_details(nl_query)
print("Structured Output:\n")
print(result)

# **  Generate SQL QUERY**
Generate a sql query using the given context and the nlp query

In [ ]:
import os
from typing import Dict, List, Optional
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model

# Define the schema for SQL query structured output.
class SQLQuerySchema(BaseModel):
    query: str = Field(..., description="The generated SQL query")

# Initialize the LLM and structured output for both table details and SQL query.
llm = init_chat_model("gpt-4o", model_provider="openai", temperature=0, openai_api_key=openai_api_key)

# Assuming get_table_details is already defined and returns a dictionary with keys "tables", "foreign_keys", etc.
# For the SQL query, create a structured output version using our SQLQuerySchema.
structured_sql_llm = llm.with_structured_output(SQLQuerySchema, method="function_calling")

# EOS_TOKEN to mark the end of the prompt output.
EOS_TOKEN = "<EOS>"

def format_sql_prompt(instruction: str, input_data: str) -> str:
    """
    Generates a standardized prompt with EOS_TOKEN that instructs the LLM to return
    only a valid SQL query based on the provided context (schema metadata) and natural language query.
    """
    sql_prompt = (
        "### Instruction:\n"
        "{instruction}\n\n"
        "### Input:\n"
        "{input}\n\n"
        "### Output (ONLY SQL query):\n\n"
        "Do NOT provide any explanation or additional text. Return only a valid SQL query "
        "that retrieves the answer based on the schema provided."
    )
    return sql_prompt.format(instruction=instruction, input=input_data) + EOS_TOKEN

def generate_sql_query(nl_query: str) -> Dict:
    """
    Uses the retrieved schema metadata (including table definitions and foreign key info)
    and the natural language query to prompt an LLM (via a RAG-based approach) to generate an SQL query.

    Args:
        nl_query (str): The natural language query describing the data requirements.

    Returns:
        dict: A structured response containing the generated SQL query with the key "query".
    """
    # Retrieve schema metadata using your existing function.
    schema_metadata = get_table_details(nl_query)  # Expected to return a dict with keys like "tables" and "foreign_keys"
    # Extract required details.
    tables = schema_metadata.get("tables_required", [])          # List of required tables.
    foreign_keys = schema_metadata.get("relationships", [])  # List of foreign key relationships.
    attributes = schema_metadata.get("attributes_required", [])    # List of required attributes.

    # Build the schema context.
    schema_context = (
        f"**Tables Required:** {', '.join(tables)}\n\n"
        f"**Attributes Required:** {', '.join(attributes)}\n\n"
        f"**Foreign Key Relationships:**\n" + ('\n'.join(foreign_keys) if foreign_keys else "None")
    )


    # Combine the schema context with the natural language query.
    input_data = f"{schema_context}\n\n**Query:** {nl_query}"

    # Build the SQL prompt with instructions to generate an SQL query.
    prompt = format_sql_prompt(
        instruction=(
            "Analyze the provided database schema and its foreign key relationships. "
            "Based on the natural language query, generate a valid SQL query that retrieves the requested data using he provided database schema and its foreign key relationships.."
        ),
        input_data=input_data
    )

    # Get the SQL query from the structured LLM.
    structured_output = structured_sql_llm.invoke(prompt)

    # Convert the Pydantic object to a dictionary.
    result_dict = structured_output.model_dump()
    return result_dict['query']

if __name__ == "__main__":
    nl_query = "which was the highest transaction users"
    sql_structured_output = generate_sql_query(nl_query)
    print("\nFinal Structured SQL Output:")
    print(sql_structured_output)


In [ ]:
# --- Example Usage ---
nl_query = "which was the highest transaction user"
result = generate_sql_query(nl_query)
print("Generated SQL Query:\n")
print(sql_structured_output['query'])


# **NLP Hallucination Check Process**
Process: Validate the generated SQL query for potential inaccuracies or hallucinations. This can involve an LLM-based review or rule-based checks before executing the query.

## **# Execute SQL Process**
Process: Run the validated SQL query against your MySQL database instance and retrieve the results.

In [ ]:
# Access the SQL query from the structured output dictionary.
sql_query_str = sql_structured_output['query']

# Execute the SQL query using the SQLDatabase instance 'db'.
result = db.run(sql_query_str)

# Print the result of the executed query.
print("Query Result:")
print(result)


# **Formulate SQL-Based Answer Process**
Process: Convert the raw SQL query results into a structured, human-readable output (e.g., JSON, tables) and optionally provide a natural language explanation.

In [ ]:
import json

def format_sql_answer(sql_result, nl_query):
    """
    Converts raw SQL query results into a structured, human-readable output,
    including a dynamic natural language explanation generated by the LLM.

    Args:
        sql_result (list or dict): The raw result from executing the SQL query.
        nl_query (str): The natural language query given as input.

    Returns:
        dict: A dictionary with a dynamic explanation and the structured data.
    """
    # Create a prompt to instruct the LLM to generate a natural language explanation.
    prompt = (
        f"Based on the following natural language query and SQL query result, "
        f"please provide a clear, concise explanation of what the result represents:\n\n"
        f"Natural Language Query: {nl_query}\n\n"
        f"SQL Query Result: {sql_result}\n\n"
        f"Explanation:"
    )

    # Use the LLM to generate the explanation dynamically.
    explanation = llm.predict(prompt)

    # Format the output in a structured dictionary.
    formatted_output = {
        "explanation": explanation,
        "natural_language_query": nl_query,
        "data": sql_result
    }

    return formatted_output

# Example usage:
# Assume `sql_structured_output` contains your SQL query and you've executed it:
sql_query_str = sql_structured_output  # Extract SQL query from the structured output.
result = db.run(sql_query_str)  # Execute the SQL query against your database.

# Format the SQL result into a human-readable output with dynamic explanation.
formatted_answer = format_sql_answer(result, nl_query)

# Print the final structured output as formatted JSON.
print(json.dumps(formatted_answer, indent=2))


# **Web Search Process**
Process: For queries routed externally, perform a web search using a tool like Tavily. Retrieve relevant news or external data that may answer the query.

In [ ]:
import os
import json
from typing import Optional
from pydantic import BaseModel, Field
from tavily import TavilyClient
from langchain.chat_models import init_chat_model

# ---------------------------
# Setup Tavily for Web Search
# ---------------------------
TAVILY_API_KEY = "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"
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

def perform_web_search(query: str) -> dict:
    """
    Performs a web search using Tavily and returns the structured search results.

    Args:
        query (str): The search query string.

    Returns:
        dict: A dictionary containing the search query and a list of results.
    """
    search_response = tavily_client.search(query)
    structured_results = {
        "query": query,
        "results": search_response.get("results", [])
    }
    return structured_results

# --------------------------------------------
# Setup LLM for Disaster News Relevance Evaluation
# --------------------------------------------
class NewsRelevanceSchema(BaseModel):
    is_relevant: bool = Field(..., description="Whether the news article is relevant to the query")
    authenticated: bool = Field(..., description="Whether the news source appears credible and authenticated")
    explanation: str = Field(..., description="Explanation for the relevance and authentication assessment")
    additional_info: Optional[str] = Field(None, description="Any additional recommendations or extra context")

llm = init_chat_model("gpt-4o", model_provider="openai", temperature=0, openai_api_key=openai_api_key)
structured_news_llm = llm.with_structured_output(NewsRelevanceSchema, method="function_calling")

def check_news_relevance(sensitive_query: str, news_article: str) -> dict:
    """
    Uses the LLM to evaluate if the provided news article content accurately
    reports on current disaster events in India and if the source appears credible.

    Args:
        sensitive_query (str): The disaster-related query.
        news_article (str): The content or snippet from the news article.

    Returns:
        dict: A structured JSON object with relevance, authentication, and explanation.
    """
    prompt = (
        "You are provided with a sensitive query regarding natural disasters in India and a news article summary. "
        "Please analyze the article and determine if it accurately reports on current disaster events in India, such as floods, landslides, or heatwaves. "
        "Evaluate whether the article is relevant to the query and if the information is credible and reliable. "
        "Provide a brief explanation for your assessment. "
        "Respond with a JSON object containing the following keys:\n"
        "  - is_relevant (bool): True if the article is relevant to the query, False otherwise.\n"
        "  - authenticated (bool): True if the news source appears credible, False otherwise.\n"
        "  - explanation (str): A brief explanation of your reasoning.\n"
        "  - additional_info (optional str): Any extra context or recommendations.\n\n"
        f"Sensitive Query: {sensitive_query}\n\n"
        f"News Article Content: {news_article}\n\n"
        "Response:"
    )
    structured_output = structured_news_llm.invoke(prompt)
    return structured_output.model_dump()

# --------------------------------
# Example Usage & Final Execution
# --------------------------------
if __name__ == "__main__":
    query = "Mention all the disasters happening currently in India this month"
    search_results = perform_web_search(query)

    if search_results.get("results"):
        first_result = search_results["results"][0]
        # Extract a snippet or content from the result (adjust the key as per Tavily's API response)
        news_article = first_result.get("snippet", first_result.get("content", ""))
        # Updated sensitive query to focus on current natural disasters
        sensitive_query = (
            "What are the latest updates on  disasters currently affecting India, "
            "including floods, landslides, and heatwaves, train collisons etc?"
        )
        relevance_info = check_news_relevance(sensitive_query, news_article)
        print(json.dumps(relevance_info, indent=2))
    else:
        print("No search results found.")


In [ ]:
# Install required packages
!pip install langgraph langsmith
!pip install langchain langchain_groq langchain_community

# Import necessary modules
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
from langgraph.graph import StateGraph
from typing import Optional, Dict, Any
from pydantic import BaseModel

class QueryState(BaseModel):
    query: str
    route: Optional[str] = None
    sql_query: Optional[Dict[str, Any]] = None
    sql_output: Optional[str] = None
    result: Optional[str] = None

In [ ]:

def router(state: QueryState):
    print(f"Routing query: {state.query}")
    state.route = "use_db" if "transaction" in state.query.lower() else "use_web"
    print(f"Route decided: {state.route}")
    return state

def use_db(state: QueryState):
    print(f"Fetching table details for query: {state.query}")
    schema_metadata = get_table_details(state.query)
    state.sql_query = schema_metadata  # Store the schema details
    return state

def generate_sql(state: QueryState):
    print(f"Generating SQL query for: {state.query}")
    sql_output = generate_sql_query(state.query)
    print(f"Generated SQL: {sql_output}")

    # Execute the SQL query
    try:
        db_result = db.run(sql_output)  # Assuming `db.run(query)` executes the SQL
        state.sql_output = db_result
        print(f"Query Execution Result: {db_result}")
    except Exception as e:
        print(f"Error executing SQL query: {e}")
        state.sql_output = str(e)

    return state


def format_answer(state: QueryState):
    print(f"Formatting answer details for query: {state.query}")
    final_output = format_sql_answer(state.sql_output,state.query)
    state.result = final_output  # Store the schema details
    print(f"Formatted Answer: {final_output}")
    return state

In [ ]:
graph_builder = StateGraph(QueryState)

graph_builder.add_node("router", router)
graph_builder.add_node("use_db", use_db)
graph_builder.add_node("generate_sql", generate_sql)
graph_builder.add_node("format_answer", format_answer)

graph_builder.set_entry_point("router")
graph_builder.add_conditional_edges(
    "router",
    lambda state: state.route,
    {
        "use_db": "use_db",
        "use_web": END,
    },
)
graph_builder.add_edge("use_db", "generate_sql")
graph_builder.add_edge("generate_sql", "format_answer")
graph_builder.set_finish_point("format_answer")

# Compile the graph
graph = graph_builder.compile()




In [ ]:
state = QueryState(query="which are llms")
output = graph.invoke(state)

# Print final output
print("Final Output:", output['result'])

In [ ]:
from IPython.display import Image, display

# Generate and display the graph
try:
    graph_image = graph.get_graph().draw_mermaid_png()
    display(Image(graph_image))
except Exception as e:
    print(f"Error generating graph: {e}")